In [102]:
import advent

data = advent.get_lines(18)

In [103]:
def magnitude(pair):
    if type(pair) == int:
        return pair
    else:
        return 3*magnitude(pair[0]) + 2*magnitude(pair[1])

def parse(pair):
    if len(pair) <= 2:
        return int(pair)
    # Looks like [xxx, yyy] where xxx/yyy may contain commas and braces
    level = 0
    for i, c in enumerate([c for c in pair[1:-1]]):
        if c == '[': level += 1
        elif c == ']': level -= 1
        elif c == ',':
            if level == 0:
                return (parse(pair[1:(i+1)]), parse(pair[(i+2):-1]))
    raise ValueError(f"{pair} could not be parsed")

test = "[[[[6,6],[7,6]],[[7,7],[7,0]]],[[[7,7],[7,7]],[[7,8],[9,9]]]]"
magnitude(parse(test))  # should be 4140


4140

In [104]:
def split(pair):
    # recurses over pair and splits when neccecary
    # Returns (pair, b) where b is boolean if we split or not

    if type(pair) == int and pair > 9:
        return (pair // 2, (pair+1) // 2), True
    elif type(pair) == int and pair <= 9:
        return pair, False
    
    left, b = split(pair[0])
    if b:
        return (left, pair[1]), b
    else:
        right, b = split(pair[1])
        return (left, right), b

def add_right(pair, num):
    if num == 0: return pair
    if type(pair) == int:
        return pair+num
    else:
        return (pair[0], add_right(pair[1], num))

def add_left(pair, num):
    if num == 0: return pair
    if type(pair) == int:
        return pair+num
    else:
        return (add_left(pair[0], num), pair[1])

def explode(pair, depth=0):
    # returns (exploded, l, r, b) where l, r still need to be added upward, b is is we split or not
    # l or r of 0 means nothing needs to be added, wether that be the inner pair was (0,0) or just no explosion
    # example: [1, 1], depth=4 returns 0, 1, 1, True
    if type(pair) == int:
        return pair, 0, 0, False
    if depth == 4:
        # Because we aggresively split, depth 5 should never be possible, so this must be some [int, int]
        assert type(pair[0]) == int and type(pair[1]) == int, "Depth higher than 4?"
        return 0, pair[0], pair[1], True

    left, l, r, b = explode(pair[0], depth+1)
    if b:
        #print(left, l, r, b, pair[1])
        # Don't explode right side. l is useless to us since this was on the left
        return (left, add_left(pair[1], r)), l, 0, b
    if not b:
        # l, r must have been 0 since there was no explosion
        right, l, r, b = explode(pair[1], depth+1)
        return (add_right(left, l), right), 0, r, b

def preduce(pair):
    exploded, splitted = True, True
    while exploded or splitted:
        pair, _, _, exploded = explode(pair)
        if exploded: continue
        pair, splitted = split(pair)
    return pair

def add(left, right):
    return preduce((left, right))

add(parse("[[[[4,3],4],4],[7,[[8,4],9]]]"), parse("[1, 1]"))





((((0, 7), 4), ((7, 8), (6, 0))), (8, 1))

In [108]:
from functools import reduce

magnitude(reduce(add, map(parse, data)))

4132

In [116]:
max_score = 0
try:
    for i in data:
        for j in data:
            max_score = max(max_score, magnitude(add(parse(i), parse(j))))
except AssertionError:
    print(i, j)
max_score

4685